In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Mar  9 23:50:23 2023

In [ ]:
@author: Baudouin
"""

%Importing Libraries

In [2]:
import pandas as pd
import numpy as np

%Importing data

In [ ]:
table_1 = pd.read_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/table_1.csv')
table_2 = pd.read_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/table_2.csv')
table_3 = pd.read_excel('/Users/Baudouin/Desktop/Chess_exploration/dataset/Table_3.xlsx')
table_4 = pd.read_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/table_4.csv')
dataset = [table_1,table_2,table_3,table_4]

%Data exploration

In [ ]:
def inital_exploration(table):
    print(table.info())
    print(table.isnull().sum())
    for col in table.columns.to_list():
        print(table[col].value_counts())
        
for table in dataset:
    print(f"###{table}###")
    inital_exploration(table)
    
#Table 1 : process date and time, encode game_type, encode outcome
#Table 2 : drop extra index col, rename columns, drop +- col, turn avg col in int
#Table 3 : compare eco codes with values in table_1 drop extra index col
#Table 4 : drop extra index col

%Table 1<br>
rop null values

In [ ]:
table_1 = table_1.dropna()
#manage date and time
table_1['year']=table_1['date'].apply(lambda x : x.split('.')[0])
table_1['month']=table_1['date'].apply(lambda x : x.split('.')[1])
table_1['day']=table_1['date'].apply(lambda x : x.split('.')[2])
table_1['hour']=table_1['time'].apply(lambda x : x.split(':')[0])
table_1['minute']=table_1['time'].apply(lambda x : x.split(':')[1])
table_1['second']=table_1['time'].apply(lambda x : x.split(':')[2])
table_1.drop(['date','time'],axis=1,inplace=True)
table_1['datetime'] = pd.to_datetime(table_1[['year', 'month', 'day', 'hour', 'minute', 'second']],
                                format='%Y-%m-%d %H:%M:%S')
table_1.drop(['year','month','day','hour','minute','second'],axis=1,inplace=True)

ncode game type

In [ ]:
def game_type(cell):
    if '/' in cell :
        gametype = 'daily'
    elif '+' in cell:
        if int(cell.split('+')[0])<600:
            gametype = 'blitz bullet'
        else :
            gametype = 'rapid bullet'
    elif int(cell)<600:
        gametype = 'blitz'
    elif int(cell)<=3600:
        gametype='rapid'
    else:
        gametype='long'
    return gametype
        
table_1['gt']=table_1['game_type']        
table_1.drop('game_type',axis=1,inplace=True)
table_1['game_type'] = table_1['gt'].apply(lambda x : game_type(x))       
table_1['game_length_in_seconds']=table_1['gt'].apply( lambda x : '-' if '/' in x else x.split('+')[0])
table_1.drop('gt',axis=1,inplace=True)   
        
#encode outcome
def outcome(outcome, player_id, opponent):
    if 'drawn' in outcome:
        gameoutcome = 0.5
    elif player_id in outcome:
        gameoutcome = 1
    elif opponent in outcome:
        gameoutcome = 0
    else:
        gameoutcome = '-'
    return gameoutcome

In [ ]:
table_1['out'] = table_1['outcome']
table_1.drop('outcome',axis=1,inplace=True)
table_1['outcome'] = table_1.apply(lambda row : outcome(row['out'],row['player_id'],row['opponent']), axis=1)
table_1['end'] = table_1['out'].apply(lambda x : x.split(' ')[-1])      
table_1.drop('out',axis=1,inplace=True)      
table_1.drop('Unnamed: 0',axis=1,inplace=True)   

ncode game moves

In [ ]:
def pgn_to_move_list(string):
    a = string.split(' { ')
    a = [i.split(' } ')[1] if ' } ' in i else i for i in a]
    a = a[:-1]
    a = [i.split('... ')[-1] if '... ' in i else i.split('. ')[-1] for i in a]
    return a

In [ ]:
table_1['moves']=table_1['game'].apply(pgn_to_move_list)

alculate the number of moves in a game

In [ ]:
table_1['moves_count']=table_1['moves'].apply(lambda x : len(x))

In [ ]:
for i in range(5):
    table_1[f"move {i+1}"] = table_1['moves'].apply(lambda x : x[i] if i<len(x) else '-')
table_1.drop(table_1['game'],inplace= True)       
       
 #%%Table 2
#dropping useless features
col_to_drop = ['Unnamed: 0',  'Avg12M']
table_2.drop(col_to_drop,axis=1,inplace=True)       
#renaming columns
new_col_name = [['name', 'fed', 'rating', 'birth_year']]      
table_2.columns =  new_col_name    
#turning avg_rating_over_12_months in an integer:

% Table 3

In [ ]:
table_1['ECO'].value_counts
a = []
table_1['ECO'].apply(lambda x : a.append(x))
a = sorted(list(set(a)))
b=[]
table_3['ECO'].apply(lambda x : b.append(x))
b = sorted(list(set(b)))
c = [i for i in a if i not in b]

% Table 4

In [ ]:
table_4.drop('Unnamed: 0', axis = 1, inplace = True)

% Save cleaned databases

In [ ]:
table_1.to_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/table_1.csv')
table_2.to_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/table_2.csv')
table_3.to_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/table_3.csv')
table_4.to_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/table_4.csv')

In [3]:
fide = pd.read_excel('/Users/Baudouin/Desktop/Chess_exploration/dataset/fide_top_100.xlsx')

In [5]:
fide.head()

,Rank,Name,Fed,Rating,B-Year,player_id
0,1,"Carlsen, Magnus",NOR,2852,1990,MagnusCarlsen
1,2,"Nepomniachtchi, Ian",RUS,2795,1990,lachesisQ
2,3,"Ding, Liren",CHN,2788,1992,Chefshouse
3,4,"Firouzja, Alireza",FRA,2785,2003,Firouzja2003
4,5,"Giri, Anish",NED,2768,1994,AnishGiri


In [6]:
a = fide['Name'].tolist()

In [7]:
a


['Carlsen, Magnus',
 'Nepomniachtchi, Ian',
 'Ding, Liren',
 'Firouzja, Alireza',
 'Giri, Anish',
 'Nakamura, Hikaru',
 'Caruana, Fabiano',
 'So, Wesley',
 'Anand, Viswanathan',
 'Radjabov, Teimour',
 'Aronian, Levon',
 'Rapport, Richard',
 'Grischuk, Alexander',
 'Dominguez Perez, Leinier',
 'Mamedyarov, Shakhriyar',
 'Vachier-Lagrave, Maxime',
 'Abdusattorov, Nodirbek',
 'Gukesh D',
 'Vidit, Santosh Gujrathi',
 'Andreikin, Dmitry',
 'Yu, Yangyi',
 'Le, Quang Liem',
 'Topalov, Veselin',
 'Maghsoodloo, Parham',
 'Duda, Jan-Krzysztof',
 'Vitiugov, Nikita',
 'Wang, Hao',
 'Wei, Yi',
 'Sjugirov, Sanan',
 'Shankland, Sam',
 'Vallejo Pons, Francisco',
 'Dubov, Daniil',
 'Harikrishna, Pentala',
 'Eljanov, Pavel',
 'Robson, Ray',
 'Artemiev, Vladislav',
 'Erigaisi Arjun',
 'Deac, Bogdan-Daniel',
 'Keymer, Vincent',
 'Sargissian, Gabriel',
 'Bu, Xiangzhi',
 'Tomashevsky, Evgeny',
 'Amin, Bassem',
 'Xiong, Jeffery',
 'Praggnanandhaa R',
 'Adams, Michael',
 'Sevian, Samuel',
 'Alekseenko, Kirill

In [8]:
a = [' '.join(x.split(', ')) if ', ' in x else x for x in a ]
a = [f"{x.split(' ')[-1]} {x.split(' ')[0:-1]}" for x in a]
a = [' '.join(x.split(', ')[::-1]).replace("'",'') for x in a]
a

['Magnus [Carlsen]',
 'Ian [Nepomniachtchi]',
 'Liren [Ding]',
 'Alireza [Firouzja]',
 'Anish [Giri]',
 'Hikaru [Nakamura]',
 'Fabiano [Caruana]',
 'Wesley [So]',
 'Viswanathan [Anand]',
 'Teimour [Radjabov]',
 'Levon [Aronian]',
 'Richard [Rapport]',
 'Alexander [Grischuk]',
 'Perez] Leinier [Dominguez',
 'Shakhriyar [Mamedyarov]',
 'Maxime [Vachier-Lagrave]',
 'Nodirbek [Abdusattorov]',
 'D [Gukesh]',
 'Santosh] Gujrathi [Vidit',
 'Dmitry [Andreikin]',
 'Yangyi [Yu]',
 'Quang] Liem [Le',
 'Veselin [Topalov]',
 'Parham [Maghsoodloo]',
 'Jan-Krzysztof [Duda]',
 'Nikita [Vitiugov]',
 'Hao [Wang]',
 'Yi [Wei]',
 'Sanan [Sjugirov]',
 'Sam [Shankland]',
 'Pons] Francisco [Vallejo',
 'Daniil [Dubov]',
 'Pentala [Harikrishna]',
 'Pavel [Eljanov]',
 'Ray [Robson]',
 'Vladislav [Artemiev]',
 'Arjun [Erigaisi]',
 'Bogdan-Daniel [Deac]',
 'Vincent [Keymer]',
 'Gabriel [Sargissian]',
 'Xiangzhi [Bu]',
 'Evgeny [Tomashevsky]',
 'Bassem [Amin]',
 'Jeffery [Xiong]',
 'R [Praggnanandhaa]',
 'Michael 

In [11]:
fide['gm_id']=fide['Name'].apply(lambda x : " ".join([x.split(',')[1],x.split(',')[0]]) if ',' in x else x)


In [14]:
fide.head()

,Rank,Fed,Rating,B-Year,player_id,gm_id
0,1,NOR,2852,1990,MagnusCarlsen,Magnus Carlsen
1,2,RUS,2795,1990,lachesisQ,Ian Nepomniachtchi
2,3,CHN,2788,1992,Chefshouse,Liren Ding
3,4,FRA,2785,2003,Firouzja2003,Alireza Firouzja
4,5,NED,2768,1994,AnishGiri,Anish Giri


In [13]:
fide = fide.drop('Name',axis=1)

In [15]:
fide.to_csv('/Users/Baudouin/Desktop/Chess_exploration/dataset/fide_top_100.csv',index = False)